In [1]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [3]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/MSFT/MSFT.csv"

In [4]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date"] = pd.to_datetime(df['date'])

df.set_index("date", inplace=True)

In [5]:
df.shape[0]

3064

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   1. open               3064 non-null   float64
 1   2. high               3064 non-null   float64
 2   3. low                3064 non-null   float64
 3   4. close              3064 non-null   float64
 4   5. adjusted close     3064 non-null   float64
 5   6. volume             3064 non-null   float64
 6   7. dividend amount    3064 non-null   float64
 7   8. split coefficient  3064 non-null   float64
dtypes: float64(8)
memory usage: 215.4 KB


In [7]:
df.head(1)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2010-01-04,30.62,31.1,30.59,30.95,23.857347,38409100.0,0.0,1.0


In [8]:
benzinga_csv_path = "/app/StockPricePredictions/data/benzinga/msft_non_dupes.csv"

In [9]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [10]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11107 entries, 0 to 11106
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      11107 non-null  object
 1   author    11107 non-null  object
 2   created   11107 non-null  object
 3   updated   11107 non-null  object
 4   title     11106 non-null  object
 5   teaser    2542 non-null   object
 6   body      9586 non-null   object
 7   url       11107 non-null  object
 8   image     11107 non-null  object
 9   channels  11107 non-null  object
 10  stocks    11107 non-null  object
 11  tags      11107 non-null  object
 12  AAPL      11107 non-null  bool  
 13  MSFT      11107 non-null  bool  
 14  AMZN      11107 non-null  bool  
 15  FB        11107 non-null  bool  
 16  NVDA      11107 non-null  bool  
 17  INTC      11107 non-null  bool  
 18  NFLX      11107 non-null  bool  
dtypes: bool(7), object(12)
memory usage: 1.1+ MB


In [11]:
df_benzinga.head(2)

,date,author,created,updated,title,teaser,body,url,image,channels,stocks,tags,AAPL,MSFT,AMZN,FB,NVDA,INTC,NFLX
0,2010-01-04,Benzinga,"Mon, 04 Jan 2010 10:05:22 -0400","Tue, 05 Jan 2010 10:02:03 -0400","Stock Market News for January 04, 2010 - Market News",NaN,"<p align=""justify"">U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. </p>\n<p align=""justify"">The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. </p>\n<p align=""justify"">This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50.</p>\n<p align=""justify"">During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express (NYSE:AXP), a 56.8% surge in Microsoft (NASDAQ:MSFT), and a 55.5% gain in IBM (NYSE:IBM). Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. </p>\n<p align=""justify"">The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday.<br />\n <br />\nOf note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.</p><a href=""http://www.zacks.com"" alt=""Investment Research"">Zacks Investment Research</a><br />",https://www.benzinga.com/node/74824,[],[],"[{'name': 'AXP'}, {'name': 'IBM'}, {'name': 'MSFT'}]","[{'name': 'Computer Hardware'}, {'name': 'Consumer Finance'}, {'name': 'Financials'}, {'name': 'Information Technology'}, {'name': 'Systems Software'}]",False,True,False,False,False,False,False
1,2010-01-04,Benzinga,"Mon, 04 Jan 2010 14:17:59 -0400","Mon, 04 Jan 2010 14:17:59 -0400","Credit Suisse AG Embroiled In A $24 Billion Lawsuit (MSFT, CS)",NaN,"According to a Reuters report, Credit Suisse AG (NYSE: <a class=""ticker"" href=""/stock/cs#NYSE"">CS</a>) has been sued by property owners for a whooping $24 billion in damages. The class action lawsuit filed in Boise, Idaho has real estate consultants Cushman & Wakefield as co defendants. \r\n\r\nThe alleged losses relate to Yellowstone resort, Lake Las Vegas resort, and Tamarack resort amongst others. According to the complaint, Credit Suisse violated federal racketeering laws by devising a ""loan to own"" scheme which inflated the value of resorts and burdened the resorts and purchasers of homes there with too much debt. The appraisal methodology devised by Cushman & Wakefield allowed Credit Suisse to win ‘enormous fees’ and take control of resorts well below market value.\r\n\r\nYellowstone, which counted Microsoft Corp 

In [12]:
df_benzinga["body"] = df_benzinga["body"].replace(np.nan, "")

In [13]:
def clean_text(text):
    soup = BeautifulSoup(text)

    # remove HTML tables
    for table in soup.find_all("table"):
        table.extract()

    text_without_tables = soup.text

    pattern1 = re.compile(r"\s")

    clean_text1 = re.sub(pattern1, " ", text_without_tables)

    pattern2 = re.compile(r"\\r|\\+n")

    clean_text2 = re.sub(pattern1, " ", clean_text1)

    clean_text3 = re.sub(r'^https?:\/\/.*[\r\n]*', '', clean_text2, flags=re.MULTILINE)

    return clean_text3

    
    

In [14]:
df_benzinga["body"] = df_benzinga["body"].apply(clean_text)

/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://allthingsd.com/20130919/microsoft-will-buy-your-old-tablets-and-smartphones/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://blogs.technet.com/b/microsoft_blog/archive/2014/03/25/microsoft-makes-source-code-for-ms-dos-and-word-for-windows-available-to-public.aspx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://blogs.technet.com/b/microsoft_on_the_issues/archive/2014/03/28/we-re-li

In [15]:
df_benzinga["body"] = df_benzinga["body"].replace(np.nan, "")

In [16]:
df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [17]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['date'])

In [18]:
df_benzinga.set_index("date", inplace=True)

In [19]:
df_benzinga = df_benzinga[["text"]]

In [20]:
def remove_ticker(text):

    clean_text1 = re.sub(r'\(NASDAQ\s?:\s?\w+\)', ' ', text, flags=re.MULTILINE)

    clean_text2 = re.sub(r'\(NYSE\s?:\s?\w+\)', ' ', clean_text1, flags=re.MULTILINE)

    return clean_text2


In [21]:
df_benzinga["text"].isnull().any()

True

In [22]:
df_benzinga["text"].fillna("", inplace=True)

In [23]:
df_benzinga["text"] = df_benzinga["text"].apply(remove_ticker)

In [24]:
benzinga_csv_out_path = "/app/StockPricePredictions/data/benzinga/msft_non_dupes_text_only.csv"

In [25]:
df_benzinga.to_csv(benzinga_csv_out_path)
df_benzinga2 = pd.read_csv(benzinga_csv_out_path)


In [26]:
df_benzinga2.head()

,date,text
0,2010-01-04,"Stock Market News for January 04, 2010 - Market News U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50. During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express , a 56.8% surge in Microsoft , and a 55.5% gain in IBM . Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday. Of note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.Zacks Investment Research"
1,2010-01-04,"Credit Suisse AG Embroiled In A $24 Billion Lawsuit (MSFT, CS) According to a Reuters report, Credit Suisse AG has been sued by property owners for a whooping $24 billion in damages. The class action lawsuit filed in Boise, Idaho has real estate consultants Cushman & Wakefield as co defendants. The alleged losses relate to Yellowstone resort, Lake Las Vegas resort, and Tamarack resort amongst others. According to the complaint, Credit Suisse violated federal racketeering laws by devising a ""loan to own"" scheme which inflated the value of resorts and burdened the resorts and purchasers of homes there with too much debt. The appraisal methodology devised by Cushman & Wakefield allowed Credit Suisse to win ‘enormous fees’ and take control of resorts well below market value. Yellowstone, which counted Microsoft Corp founder Bill Gates amongst its patrons, had pleaded for bankruptcy last May and the judge had castigated Credit Suisse for ‘predatory lending practices’. Judge Ralph Rischer who oversaw the bankruptcy proceedings went so far as to state ‘The naked greed in this case shocks the conscience of this court’."
2,2010-01-04,"Technology Stocks To Outperform In 2010 (STX, 3PAR, EMC, IBM, MSFT, AAPL) With companies starting to devote much more resources to technology to enhance productivity of existing workers, GDP growth of 2-3% should lead to an IT spending rebound of 5%+ in 2010, according to Hapolim Securities. In 2009, the HDD (hard disk drive) sub segment posted returns of 41.2% versus a NASDAQ gain of 17.5% while Storage systems were a close second with 34.5% return. Strong expected PC demand and continued tight capacity will be the main success drivers in 2010 for HDD, a se

In [27]:
# import spacy

In [28]:
# nlp = spacy.load('en_core_web_md')
# myvals = df_benzinga['text'].values.tolist()

In [29]:
# df_benzinga['len'] = df_benzinga['text'].str.split().str.len()

In [30]:
# df_benzinga.describe()